In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
from sqlalchemy.engine import create_engine
output_schema_name = 'presence_data'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [4]:
import time
import pandas as pd
from sqlalchemy import create_engine
from trino.dbapi import connect
from datetime import datetime, timedelta

In [5]:
%%sql
show schemas from dedicated

 * trino://localhost:9090/cuebiq/
Done.


,Schema
0,censustocensus
1,dataflowtest
2,default
3,ds_data_for_complexity
4,ds_data_for_complexity_v2
5,ds_data_for_complexity_v3
6,information_schema
7,od_matrix
8,od_matrix20
9,od_matrix_10


## Create table 

In [7]:
# Define the table name
country_code = 'CO'
# final_table = f"od52_{country_code.lower()}_2019_3h"
final_table = f"od3_{country_code.lower()}_2019_3h"

output_schema_name = "od_matrix_10"
# output_schema_name = 'presence_data'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

            
# Create the SQL table with the correct columns
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {final_table} (
    start_geohash5 varchar,
    end_geohash5 varchar,
    local_date bigint,
    trip_count bigint,
    m_duration_min double,
    mdn_duration_min double,
    sd_duration_min double,
    m_length_m double,
    mdn_length_m double,
    sd_length_m double,
    m_points_no double,
    mdn_points_no double,
    sd_points_no double,
    local_time varchar
)
"""

# Connect to the SQL server and execute the create table query
with con.connect() as connection:
    connection.execute(create_table_query)


## Insert table 

In [9]:
file = '/home/jovyan/Data/TJ/3h/done/od_co_agg3_3h.csv'
final_df = pd.read_csv(file)

# # Set the correct dtypes
# final_df = final_df.astype({
#     'start_geohash5': 'str',
#     'end_geohash5': 'str',
#     'local_date': 'int',
#     'trip_count': 'int',
#     'm_duration_min': 'float',
#     'mdn_duration_min': 'float',
#     'sd_duration_min': 'float',
#     'm_length_m': 'float',
#     'mdn_length_m': 'float',
#     'sd_length_m': 'float',
#     'm_points_no': 'float',
#     'mdn_points_no': 'float',
#     'sd_points_no': 'float',
#     'local_time': 'str'
# })

final_df = final_df.astype({
    'start_geohash3': 'str',
    'end_geohash3': 'str',
    'local_date': 'int',
    'trip_count': 'int',
    'm_duration_min': 'float',
    'mdn_duration_min': 'float',
    'sd_duration_min': 'float',
    'm_length_m': 'float',
    'mdn_length_m': 'float',
    'sd_length_m': 'float',
    'm_points_no': 'float',
    'mdn_points_no': 'float',
    'sd_points_no': 'float',
    'local_time': 'str'
})

# test = final_df.head(5)
test = final_df.tail(28)
# test.dtypes
# final_df.dtypes
test
# final_df

,start_geohash3,end_geohash3,local_date,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_time
14500,d22,d22,20191231,10,30.383333,25.425000,24.655705,706.456438,550.423478,493.957673,5.800000,5.0,3.359894,20191231 21:00:00 - 24:00:00
14501,d23,d23,20191231,79,32.690295,25.450000,26.003804,4299.648686,906.515947,10356.409808,4.696203,4.0,3.585421,20191231 21:00:00 - 24:00:00
14502,d26,d26,20191231,52,26.213141,16.333333,23.029274,3452.934397,843.012654,7157.500567,4.326923,4.0,2.572259,20191231 21:00:00 - 24:00:00
14503,d29,d29,20191231,624,33.676950,23.216667,31.649313,3529.389510,1264.745712,7340.799272,6.040064,4.0,15.841206,20191231 21:00:00 - 24:00:00
14504,d2c,d2c,20191231,45,33.645926,26.500000,26.053373,1320.455622,611.573167,2556.693299,4.666667,4.0,2.688359,20191231 21:00:00 - 24:00:00
14505,d2d,d2d,20191231,170,34.028627,23.375000,30.114493,2683.176591,965.197227,5585.650816,4.429412,4.0,2.424789,20191231 21:00:00 - 24:00:00
14506,d2e,d2e,20191231,164,35.967073,24.191667,33.728599,3114.476351,970.572178,7197.915602,4.487805,4.0,2.473139,20191231 21:00:00 - 24:00:00
14507,d2f,d2f,20191231,708,30.852448,23.166667,25.476387,2994.330689,905.733159,7338.675558,5.416667,4.0,11.170283,20191231 21:00:00 - 24:00:00
14508,d2g,d2g,20191231,1299,29.780883,20.650000,27.978824,4492.823831,641.395354,20337.818484,5.318707,4.0,4.367556,20191231 21:00:00 - 24:00:00
14509,d2k,d2k,20191231,14,13.904762,9.150000,11.381570,1412.877172,481.098439,1924.419108,4.785714,4.0,2.293086,20191231 21:00:00 - 24:00:00


In [10]:
test.to_sql(final_table, con, if_exists='append', index=False, method='multi', chunksize = 500)
# final_df.to_sql(final_table, con, if_exists='append', index=False, method='multi')

Http502Error: error 502: bad gateway

# Check output

In [11]:
q1 = f"select * from dedicated.{output_schema_name}.{final_table}"

%sql $q1

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash3,end_geohash3,local_date,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_time
0,d2t,d2t,20191120,10,137.393333,72.091667,191.178702,1698.078264,576.022845,2228.518676,7.500000,6.0,8.343327,20191120 06:00:00 - 09:00:00
1,d2u,d2u,20191120,134,101.197512,37.183333,154.056330,5212.456660,1262.756708,12502.610325,5.582090,4.0,5.241963,20191120 06:00:00 - 09:00:00
2,d31,d31,20191120,53,102.012893,46.733333,144.829295,3195.368861,1286.404821,8333.044695,5.320755,4.0,3.750762,20191120 06:00:00 - 09:00:00
3,d33,d33,20191120,69,100.650242,47.550000,147.690978,4778.215333,1282.262611,11235.886768,4.579710,4.0,2.897290,20191120 06:00:00 - 09:00:00
4,d34,d34,20191120,2318,116.494485,49.833333,170.468064,6435.319217,2606.963376,14505.023662,7.120362,5.0,6.818297,20191120 06:00:00 - 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14495,d3h,d3h,20191213,13,114.261538,26.983333,178.615961,1953.136090,515.313695,4029.181772,4.307692,4.0,1.974192,20191213 00:00:00 - 03:00:00
14496,d3k,d3k,20191213,48,391.191319,256.175000,394.620410,6480.204560,1806.917320,11936.261683,6.958333,5.0,7.411774,20191213 00:00:00 - 03:00:00
14497,d20,d20,20191213,10,64.278333,38.816667,89.536445,1209.319957,1123.814730,794.357739,5.300000,4.5,3.020302,20191213 03:00:00 - 06:00:00
14498,d21,d21,20191213,13,293.960256,61.066667,366.892519,11282.065012,1083.101863,26099.562090,9.769231,5.0,11.144160,20191213 03:00:00 - 06:00:00


In [10]:
# import pandas as pd
# df = pd.read_csv('/home/jovyan/Data/Agg_DL/ID3/20190101_ID_pe_dl_agg3.csv')
# df

,geohash_3,point_number,user_number
0,qpx,1255,43
1,qpz,46864,845
2,qqe,1309,48
3,qqg,1981239,22776
4,qqs,57836,1189
...,...,...,...
132,wb2,34656,622
133,wb3,1331,54
134,wb4,3246,87
135,wb7,670,28


In [18]:
# %%time
# df.to_sql(
#     "my_processed_dataset3", 
#     con, 
#     index=False, 
#     if_exists="append", 
#     method="multi"
# )

CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 2.37 s


137

1. if_exists='fail': The method will raise a ValueError if the table already exists. This is the default behavior.
2. if_exists='replace': If the table exists, it will be dropped and replaced with the new data.
3. if_exists='append': If the table exists, the new data will be inserted into the existing table. If the table does not exist, it will be created.

 * trino://localhost:9090/cuebiq/
Done.


,geohash_5,point_number,user_number
0,t9qxx,1681,32
1,t9qxz,1708,54
2,t9qyq,344,12
3,t9qyr,657,18
4,t9qyt,957,18
...,...,...,...
10947,whguy,408,21
10948,whgwq,497,14
10949,whu5b,645,15
10950,whujm,381,12


In [ ]:
df = pd.read_csv('/home/jovyan/Data/DL/MX/20190128_MX_pe_dl.csv')
# df.sort_values('event_datetime_local')
df

In [3]:
import gc
import os
import time
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [4]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [5]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 

In [6]:
class TrinoEngine:
    def __init__(self):
        self.conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = self.conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query: str):
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql_chunked(self, query: str, chunksize: int = 10000):
        return pd.read_sql(query, self.engine, chunksize=chunksize)

sql_engine = TrinoEngine()


In [39]:
country_code = 'ID'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')
formatted_date = start_date_dt.strftime('%Y%m%d')
next_date = (start_date_dt + timedelta(days=1)).strftime('%Y-%m-%d')

In [42]:
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
    FROM {pe_dl_table}
    WHERE 
        processing_date = {formatted_date} 
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
    """
)

# pe_tj_table
first_chunk = next(pe_tj_table)
first_chunk

,cuebiq_id,event_zoned_datetime,processing_date,lat,lng,event_datetime_local
0,1514830569,2019-01-01T20:09:18+07:00,20190101,-7.322733,108.222739,2019-01-01 20:09:18
1,1514830569,2019-01-01T20:05:16+07:00,20190101,-7.319290,108.220144,2019-01-01 20:05:16
2,1514830569,2019-01-01T20:13:38+07:00,20190101,-7.319290,108.220144,2019-01-01 20:13:38
3,1514830569,2019-01-01T20:17:39+07:00,20190101,-7.312219,108.211619,2019-01-01 20:17:39
4,1514830569,2019-01-01T19:59:15+07:00,20190101,-7.306623,108.206429,2019-01-01 19:59:15
...,...,...,...,...,...,...
9995,1513028254,2019-01-01T08:22:51+07:00,20190101,-6.525583,106.854246,2019-01-01 08:22:51
9996,1513028254,2019-01-01T05:32:23+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:23
9997,1513028254,2019-01-01T05:32:23+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:23
9998,1513028254,2019-01-01T05:32:24+07:00,20190101,-6.525322,106.854464,2019-01-01 05:32:24


In [31]:
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM {pe_dl_table}
    WHERE 
        processing_date = {formatted_date} 
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)

pe_tj_table_df

# Process date 用前面的

ProgrammingError: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 11:10: mismatched input '<'. Expecting: '(', 'LATERAL', 'TABLE', 'UNNEST', <identifier>", query_id=20240604_181830_00347_tt5ya)
[SQL: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM <generator object SQLDatabase._query_iterator at 0x7fecf51b2430>
    WHERE 
        processing_date = 20190101 
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('20190101', '%Y%m%d'

In [16]:
# probabliy not correct!!!!
# 因为要求时间<开始时间

from datetime import datetime, timedelta
import pandas as pd

import time

# Start timing
start_time = time.time()

# Define the input parameters
country_code = 'ID'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Calculate the date 45 days before the start_date
lookback_date_dt = start_date_dt - timedelta(days=45)
formatted_lookback_date = lookback_date_dt.strftime('%Y%m%d')
formatted_end_date = end_date_dt.strftime('%Y%m%d')

# Construct the SQL query
pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng, 
        TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
        DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

    FROM {pe_dl_table}
    WHERE 
        processing_date >= {formatted_lookback_date}
        AND processing_date <= {formatted_end_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{end_date_dt.strftime('%Y-%m-%d')}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)


# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")

pe_tj_table_df

# probabliy not correct!!!!

KeyboardInterrupt: 

In [7]:
# This one seems like correct 
from datetime import datetime, timedelta
import pandas as pd
import time

# Start timing
start_time = time.time()

# Define the input parameters
country_code = 'MX'
start_date = 20190102
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct and execute the SQL query
    pe_dl_table = sql_engine.read_sql_chunked(
        f"""
        SELECT 
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            lat,
            lng, 
            TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local,
            DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) AS event_date_local

        FROM {pe_dl_table}
        WHERE 
            processing_date >= {formatted_lookback_date} 
            AND processing_date <= {formatted_lookahead_date}
            AND country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
            AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
            AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
        """
    )
    
    # Convert the generator to a DataFrame
    chunks = [chunk for chunk in pe_tj_table]
    if chunks:
        pe_tj_table_df = pd.concat(chunks, ignore_index=True)
        
        # Save the DataFrame to a CSV file
        output_filename = f'/home/jovyan/Data/Test/0604/pe_tj_table_{formatted_current_date}.csv'
        pe_tj_table_df.to_csv(output_filename, index=False)
        print(f"Saved data for {formatted_current_date} to {output_filename}")
    
    # Move to the next day
    current_date += timedelta(days=1)

print("Data extraction and saving completed.")


end_time = time.time() # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


Saved data for 20190102 to /home/jovyan/Data/Test/0604/pe_tj_table_20190102.csv
Saved data for 20190103 to /home/jovyan/Data/Test/0604/pe_tj_table_20190103.csv
Data extraction and saving completed.
Total time taken: 1972.0941424369812 seconds


In [8]:
# transform all event date and get one day. 
import time

country_code = 'MX'
start_date = 20190101

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
next_date = (start_date_dt + timedelta(days=1)).strftime('%Y-%m-%d')

# Start timing
start_time = time.time()

pe_tj_table = sql_engine.read_sql_chunked(
    f"""
    WITH event_data AS (
        SELECT 
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            timezoneoffset_secs,
            lat,
            lng, 
            TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
        FROM {pe_dl_table}
        WHERE 
            country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
            AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
    )
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        timezoneoffset_secs,
        lat,
        lng
    FROM event_data
    WHERE 
        event_datetime_local >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
        AND event_datetime_local < date_parse('{next_date}', '%Y-%m-%d')
    """
)

# Convert the generator to a DataFrame
chunks = [chunk for chunk in pe_tj_table]
pe_tj_table_df = pd.concat(chunks, ignore_index=True)

# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


pe_tj_table_df


# transform all event date and get one day. 
# 877 for one day in ID 
# 1980 for one day in MX

Total time taken: 1980.333106994629 seconds


,cuebiq_id,event_zoned_datetime,processing_date,timezoneoffset_secs,lat,lng
0,1698771180,2019-01-01T07:54:30-06:00,20190101,-21600,17.992812,-93.373950
1,1698771180,2019-01-01T15:56:44-06:00,20190101,-21600,18.000913,-93.371119
2,1698770637,2019-01-01T15:14:05-06:00,20190101,-21600,22.761220,-102.537551
3,1698770637,2019-01-01T02:11:25-06:00,20190101,-21600,22.761236,-102.537586
4,1698770637,2019-01-01T01:03:56-06:00,20190101,-21600,22.761512,-102.537348
...,...,...,...,...,...,...
20410846,1133006951,2019-01-01T16:18:02-06:00,20190127,-21600,24.035794,-104.652039
20410847,1133006951,2019-01-01T16:38:48-06:00,20190127,-21600,24.036156,-104.652051
20410848,1133006951,2019-01-01T17:06:00-06:00,20190127,-21600,24.036156,-104.652051
20410849,1133006951,2019-01-01T18:08:46-06:00,20190127,-21600,24.036297,-104.651470


In [11]:
pip install geohash2

  Using cached geohash2-1.1-py3-none-any.whl
  Using cached docutils-0.21.2-py3-none-any.whl (587 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install python-geohash

Note: you may need to restart the kernel to use updated packages.


In [5]:
# with logging and works. export data to the system. 
# 1933.101960659027 for 3 day in IN
# process date -1 to +35
from datetime import datetime, timedelta
import pandas as pd
import time
import geohash2
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

start_time = time.time()  # Start timing

# Define the input parameters
country_code = 'IN'
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct the SQL query
    query = f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM {pe_dl_table}
    WHERE 
        processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
    """
    
    logging.info(f"Executing SQL query for date {formatted_current_date}: {query}")
    
    try:
        pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
        # Convert the generator to a DataFrame
        chunks = [chunk for chunk in pe_dl_table_gen]
        if chunks:
            pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
            # Calculate geohashes
            pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
            pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
            # Aggregate data for geohash5
            aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
                no_of_points=('geohash5', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Save the aggregated data to a CSV file for geohash5
            output_filename_5 = f'/home/jovyan/Data/Agg_DL/IN5/aggregated_pe_tj_table_geohash5_{formatted_current_date}.csv'
            aggregated_data_5.to_csv(output_filename_5, index=False)
            logging.info(f"Saved aggregated data for geohash5 for {formatted_current_date} to {output_filename_5}")

            # Aggregate data for geohash3
            aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
                no_of_points=('geohash3', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Save the aggregated data to a CSV file for geohash3
            output_filename_3 = f'/home/jovyan/Data/Agg_DL/IN3/aggregated_pe_tj_table_geohash3_{formatted_current_date}.csv'
            aggregated_data_3.to_csv(output_filename_3, index=False)
            logging.info(f"Saved aggregated data for geohash3 for {formatted_current_date} to {output_filename_3}")
        
    except Exception as e:
        logging.error(f"Error while processing data for date {formatted_current_date}: {e}")
    
    # Move to the next day
    current_date += timedelta(days=1)

logging.info("Data extraction, aggregation, and saving completed.")

end_time = time.time()  # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
logging.info(f"Total time taken: {total_time} seconds")


2024-06-04 18:47:51,936 - INFO - Executing SQL query for date 20190101: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM cuebiq.paas_cda_pe_v3.device_location_uplevelled
    WHERE 
        processing_date BETWEEN 20181231 AND 20190205
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('20190101', '%Y%m%d')
    
2024-06-04 18:57:53,341 - INFO - Saved aggregated data for geohash5 for 20190101 to /home/jovyan/Data/Agg_DL/IN3/aggregated_pe_tj_table_geohash5_20190101.csv
2024-06-04 18:57:55,233 - INFO - Saved aggregated data for geohash3 for 20190101 to /home/jovyan/Data/Agg_DL/IN5/aggregated_pe_tj_table_geohash3_20190101.csv
2024-06-04 18:57:55,234 - INFO - Executing SQL query for date 20190

In [7]:
# Write in to the table

from datetime import datetime, timedelta
import pandas as pd
import time
import geohash2
import logging
from sqlalchemy import create_engine

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'presence_data'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

start_time = time.time()  # Start timing

# Define the input parameters
country_code = 'IN'
start_date = 20190101
end_date = 20190103

# Define the input schema and table name
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    # Calculate the lookback and lookahead dates
    lookback_date = current_date - timedelta(days=1)
    lookahead_date = current_date + timedelta(days=35)
    
    # Format dates for the SQL query
    formatted_lookback_date = lookback_date.strftime('%Y%m%d')
    formatted_current_date = current_date.strftime('%Y%m%d')
    formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
    
    # Construct the SQL query
    query = f"""
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM {pe_dl_table}
    WHERE 
        processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
        AND country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
    """
    
    logging.info(f"Executing SQL query for date {formatted_current_date}: {query}")
    
    try:
        pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
        # Convert the generator to a DataFrame
        chunks = [chunk for chunk in pe_dl_table_gen]
        if chunks:
            pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
            # Calculate geohashes
            pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
            pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
            # Aggregate data for geohash5
            aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
                no_of_points=('geohash5', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Insert aggregated data for geohash5 into SQL table
            table_name_agg5 = f"pd_{country_code}_{formatted_current_date}_agg5"
            insert_data_with_retry(aggregated_data_5, table_name_agg5, con)
            
            # Aggregate data for geohash3
            aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
                no_of_points=('geohash3', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Insert aggregated data for geohash3 into SQL table
            table_name_agg3 = f"pd_{country_code}_{formatted_current_date}_agg3"
            insert_data_with_retry(aggregated_data_3, table_name_agg3, con)
        
    except Exception as e:
        logging.error(f"Error while processing data for date {formatted_current_date}: {e}")
    
    # Move to the next day
    current_date += timedelta(days=1)

logging.info("Data extraction, aggregation, and saving completed.")

end_time = time.time()  # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
logging.info(f"Total time taken: {total_time} seconds")


2024-06-04 19:34:15,684 - INFO - Executing SQL query for date 20190101: 
    SELECT 
        cuebiq_id, 
        event_zoned_datetime, 
        processing_date,
        lat,
        lng
    FROM cuebiq.paas_cda_pe_v3.device_location_uplevelled
    WHERE 
        processing_date BETWEEN 20181231 AND 20190205
        AND country_code = 'IN' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('20190101', '%Y%m%d')
    
/tmp/ipykernel_723/3963072833.py:36: UserWarning: The provided table name 'pd_IN_20190101_agg5' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(
2024-06-04 19:49:26,410 - INFO - Inserted data into table pd_IN_20190101_agg5
/tmp/ipykernel_723/3963072833.py:36: UserWarni

KeyboardInterrupt: 